# Total / Average Emissions Benchmark
## Comparing the results of NEMED to AEMO CDEII Reporting
Using the package <code>nemed</code> to calculate daily total NEM emissions, and average emissions (or emissions intensity) over a historical period.

### Data Preparation
#### Import Packages

In [ ]:
import nemed
from nemed.downloader import download_current_aemo_cdeii_summary

# To generate plots shown
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

#### Define Parameters

In [ ]:
cache = "E:/TEMPCACHE/" # Set this to an existing empty folder or leave as ""
                        # to create a new cache folder
start_time = "2021/07/01 00:00:00" # Define the start of the historical period.
end_time = "2022/07/01 00:00:00" # Define the end of the historical period.
filter_units = None # List DUIDs if you only require data for specific units.
filter_regions = None # List regions if you only require data for such.

#### Get data using NEMED

In [ ]:
result = nemed.get_total_emissions(start_time, end_time, cache, filter_regions,
                                   by="day", generation_sent_out=True)

##### Sent Out Energy Generation

In [ ]:
energy = result['Energy']
energy

##### Total Emissions

In [ ]:
emissions_tot = result['Total_Emissions']
emissions_tot

##### Emissions Intensity (Average Emissions)

In [ ]:
emissions_avg = result['Intensity_Index']
emissions_avg

##### Total Metrics of NEM

In [ ]:
energy['NEM'] = energy[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_tot['NEM'] = emissions_tot[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_avg['NEM'] = emissions_tot['NEM'] / energy['NEM']

#### Retrieve Published AEMO CDEII data for comparison

In [ ]:
aemo = download_current_aemo_cdeii_summary(start_time, end_time) # CONFIGURED FOR 2021-2022 DATA
aemo

#### Compute Error between Datasets

In [ ]:
def calculate_total_emissions_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = [emissions_tot[region].values[i] - sel_aemo['TOTAL_EMISSIONS'][i] for i in range(min(len(emissions_tot[region]), len(sel_aemo['TOTAL_EMISSIONS']))-1)]
    error_pct = [(error_calc[i] / sel_aemo['TOTAL_EMISSIONS'][i]) * 100 for i in range(len(error_calc)) if sel_aemo['TOTAL_EMISSIONS'][i]>10]
    return error_pct

def calculate_energy_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc_en = [energy[region].values[i] - sel_aemo['TOTAL_SENT_OUT_ENERGY'][i] for i in range(min(len(energy[region]), len(sel_aemo['TOTAL_SENT_OUT_ENERGY']))-1)]
    error_pct_en = [(error_calc_en[i] / sel_aemo['TOTAL_SENT_OUT_ENERGY'][i]) * 100 for i in range(len(error_calc_en))]
    return error_pct_en

def calculate_intensity_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = [emissions_avg[region].values[i] - sel_aemo['CO2E_INTENSITY_INDEX'][i] for i in range(min(len(emissions_avg[region]), len(sel_aemo['CO2E_INTENSITY_INDEX']))-1)]
    error_pct = [(error_calc[i] / sel_aemo['CO2E_INTENSITY_INDEX'][i]) * 100 for i in range(len(error_calc))]
    return error_pct

#### Setup Plotting
##### Energy Plot Format

In [ ]:
def plot_energy(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct_en = calculate_energy_error(region)

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]])
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Sent Out Generation<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Energy (MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_SENT_OUT_ENERGY'],\
        name="AEMO CDEII Reported",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=energy[region].values,\
        name="nswED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=error_pct_en,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Total Emissions Plot Format

In [ ]:
def plot_emissions(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_total_emissions_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Total Emissions<br><sub>nsw Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_EMISSIONS'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index,y=emissions_tot[region].values,\
        name="nswED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Emissions Intensity (Average Emissions) Plot Format

In [ ]:
def plot_intensity(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_intensity_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Average Emissions (Emissions Intensity)<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e/MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['CO2E_INTENSITY_INDEX'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index,y=emissions_avg[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines", line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

### Results - Comparison to AEMO

#### NEM

In [ ]:
region = 'NEM'
plot_energy(region, color_idx=0)
plot_emissions(region, color_idx=0)
plot_intensity(region, color_idx=0)

```{include} charts_ex1/energy_nem.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nem.html)

```{include} charts_ex1/emissions_nem.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nem.html)

```{include} charts_ex1/intensity_nem.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nem.html)

#### NSW

In [ ]:
region = 'NSW1'
plot_energy(region, color_idx=1)
plot_emissions(region, color_idx=1)
plot_intensity(region, color_idx=1)

```{include} charts_ex1/energy_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nsw.html)

```{include} charts_ex1/emissions_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nsw.html)

```{include} charts_ex1/intensity_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nsw.html)

#### VIC

In [ ]:
region = 'VIC1'
plot_energy(region, color_idx=3)
plot_emissions(region, color_idx=3)
plot_intensity(region, color_idx=3)

```{include} charts_ex1/energy_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_vic.html)

```{include} charts_ex1/emissions_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_vic.html)

```{include} charts_ex1/intensity_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_vic.html)

#### QLD

In [ ]:
region = 'QLD1'
plot_energy(region, color_idx=2)
plot_emissions(region, color_idx=2)
plot_intensity(region, color_idx=2)

```{include} charts_ex1/energy_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_qld.html)

```{include} charts_ex1/emissions_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_qld.html)

```{include} charts_ex1/intensity_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_qld.html)

#### SA

In [ ]:
region = 'SA1'
plot_energy(region, color_idx=4)
plot_emissions(region, color_idx=4)
plot_intensity(region, color_idx=4)

```{include} charts_ex1/energy_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_sa.html)

```{include} charts_ex1/emissions_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_sa.html)

```{include} charts_ex1/intensity_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_sa.html)

#### TAS

In [ ]:
region = 'TAS1'
plot_energy(region, color_idx=5)
plot_emissions(region, color_idx=5)
plot_intensity(region, color_idx=5)

```{include} charts_ex1/energy_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_tas.html)

```{include} charts_ex1/emissions_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_tas.html)

```{include} charts_ex1/intensity_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_tas.html)